In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression

In [2]:
# load dataframe 
bikes_df = pd.read_pickle('cleaned_data/bikes_regression.pkl')